Here I am just coding the model, you can copy the other modules like dataset creation and training from the 3dCNN code [here](https://github.com/khetansarvesh/video/blob/main/action_recognision/3DCNN.ipynb)!!

In [ ]:
class Resnet_LSTM_Model(nn.Module):

    def __init__(self):
        super(Resnet_LSTM_Model, self).__init__()

        # ResNet
        resnet = models.resnet152(pretrained=True)
        modules = list(resnet.children())[:-1]      # delete the last fc layer of resent architecture
        self.resnet = nn.Sequential(*modules)

        # FFNN
        self.ffn1 = nn.Sequential(
                                    nn.Linear(resnet.fc.in_features, 1024), nn.BatchNorm1d(1024, momentum=0.01), nn.ReLU(),
                                    nn.Linear(1024, 768), nn.BatchNorm1d(768, momentum=0.01), nn.ReLU(),
                                    nn.dropout(0.3),
                                    nn.Linear(768, 512)
                                )

        self.LSTM = nn.LSTM( input_size=512, hidden_size=512, num_layers=3, batch_first=True)


        self.ffn2 = nn.Sequential(
                                  nn.Linear(512, 256), nn.ReLU(),
                                  nn.Linear(256, 101)
                                  )


    def forward(self, x_3d):

        cnn_embed_seq = []
        for t in range(x_3d.size(1)):

            with torch.no_grad():

                # Resnet
                x = self.resnet(x_3d[:, t, :, :, :])

                # Flatten
                x = x.view(x.size(0), -1)

            # FFNN
            x = self.ffn1(x)
            cnn_embed_seq.append(x)

        # swap time and sample dim such that (sample dim, time dim, CNN latent dim)
        cnn_embed_seq = torch.stack(cnn_embed_seq, dim=0).transpose_(0, 1) # shape => (batch, time_step, input_size)

        # passing through LSTM Layer
        self.LSTM.flatten_parameters()
        lstm_output, (h_n, h_c) = self.LSTM(cnn_embed_seq, None)  # shape_lstm_output => (batch, time_step, output_size), shape_h_n => (n_layers, batch, hidden_size), shape_h_c => (n_layers, batch, hidden_size)

        # passing through FFNN Layer
        final_output = self.fnn2(lstm_output[:, -1, :])   # choose x at the last time step

        return final_output